# Notebook Description
In this notebook I've used Gemma-2-9b-it model to paraphrase the math problem classification data from
[this](https://www.kaggle.com/competitions/classification-of-math-problems-by-kasut-academy/overview) Kaggle competition.

### What this Notebook has:
- Uses vllm to load the model.
- Run inference to generate paraphrased data from the original data.
- Save the data.

### Next steps for this exploration:
- Better analysis of the original vs the paraphrased data.
- Analyze how much of the actual meaning is carried over to the paraphrased version.

# Install Packages

In [1]:
!pip install vllm -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.1/294.1 MB 5.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 89.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 1.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 94.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 95.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 40.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 96.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1

# Import Libraries

In [ ]:
import pandas as pd
from vllm import LLM, SamplingParams

INFO 04-25 16:07:21 [__init__.py:239] Automatically detected platform cuda.


2025-04-25 16:07:23.517765: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745597243.720718      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745597243.780592      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Load Quantized Gemma Model

In [ ]:
llm = LLM(model="hugging-quants/gemma-2-9b-it-AWQ-INT4", dtype="float16", quantization='awq', tensor_parallel_size=2)

config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

WARNING 04-25 16:09:49 [config.py:2836] Casting torch.bfloat16 to torch.float16.
INFO 04-25 16:10:02 [config.py:689] This model supports multiple tasks: {'classify', 'reward', 'score', 'generate', 'embed'}. Defaulting to 'generate'.
WARNING 04-25 16:10:03 [config.py:768] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
WARNING 04-25 16:10:03 [arg_utils.py:1731] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
INFO 04-25 16:10:03 [config.py:1713] Defaulting to use mp for distributed inference
INFO 04-25 16:10:03 [llm_engine.py:243] Initializing a V0 LLM engine (v0.8.4) with config: model='hugging-quants/gemma-2-9b-it-AWQ-INT4', speculative_config=None, tokenizer='hugging-quants/gemma-2-9b-it-AWQ-INT4', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_forma

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/87.1k [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


(VllmWorkerProcess pid=151) INFO 04-25 16:11:19 [weight_utils.py:281] Time spent downloading weights for hugging-quants/gemma-2-9b-it-AWQ-INT4: 3.121956 seconds
INFO 04-25 16:11:21 [loader.py:458] Loading weights took 5.98 seconds
INFO 04-25 16:11:22 [model_runner.py:1146] Model loading took 2.8943 GiB and 20.748736 seconds
(VllmWorkerProcess pid=151) INFO 04-25 16:11:22 [loader.py:458] Loading weights took 3.64 seconds
(VllmWorkerProcess pid=151) INFO 04-25 16:11:23 [model_runner.py:1146] Model loading took 2.8943 GiB and 21.611032 seconds
(VllmWorkerProcess pid=151) INFO 04-25 16:11:37 [worker.py:267] Memory profiling takes 14.32 seconds
(VllmWorkerProcess pid=151) INFO 04-25 16:11:37 [worker.py:267] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.90) = 13.27GiB
(VllmWorkerProcess pid=151) INFO 04-25 16:11:37 [worker.py:267] model weights take 2.89GiB; non_torch_memory takes 0.12GiB; PyTorch activation peak memory takes 0.66GiB; the rest of t

Capturing CUDA graph shapes:   0%|          | 0/35 [00:00<?, ?it/s]

(VllmWorkerProcess pid=151) INFO 04-25 16:11:43 [model_runner.py:1456] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilization` or switching to eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 04-25 16:12:32 [custom_all_reduce.py:195] Registering 2975 cuda graph addresses
(VllmWorkerProcess pid=151) INFO 04-25 16:12:32 [custom_all_reduce.py:195] Registering 2975 cuda graph addresses
(VllmWorkerProcess pid=151) INFO 04-25 16:12:32 [model_runner.py:1598] Graph capturing finished in 49 secs, took 0.43 GiB
INFO 04-25 16:12:32 [model_runner.py:1598] Graph capturing finished in 49 secs, took 0.43 GiB
INFO 04-25 16:12:32 [llm_engine.py:449] init engine (profile, create kv cache, warmup model) took 69.52 seconds


# Load Data

In [5]:
train = pd.read_csv("/kaggle/input/classification-of-math-problems-by-kasut-academy/train.csv")
test = pd.read_csv("/kaggle/input/classification-of-math-problems-by-kasut-academy/test.csv")

# Create prompts for training data

In [ ]:
sampling_params_pp = SamplingParams(max_tokens=1000)

prompt_template_pp = """You are a an excellent paraphrasing AI assistant. Given a math problem in latex, you will remove unnecessary text like links from it and paraphrase it without changing the actual meaning of the question.

Math Problem:
{problem}

Instructions:
**You must only return the paraphrased version of the given math problem and nothing else in your response.**

Answer:"""

prompts_pp = [prompt_template_pp.format(problem=q) for q in train["Question"]]

print(prompts_pp[-1])

You are a an excellent paraphrasing AI assistant. Given a math problem in latex, you will remove unnecessary text like links from it and paraphrase it without changing the actual meaning of the question.

Math Problem:
4. In a football tournament, only teams from Small Town and Big City participated. There were 9 more teams from Big City than from Small Town. Each team met exactly once, with the winning team getting 1 point, the losing team 0 points, and no draws were possible. Teams from Big City scored 9 times as many points as teams from Small Town. Determine the maximum possible number of wins for the best team from Small Town.

The task should be solved independently. You have 210 minutes for solving. The use of notes, literature, or a pocket calculator is not allowed.

49th Mathematical Competition for High School Students in Slovenia

Velenje, April 16, 2005

## Tasks for 4th Year

Instructions:
**You must only return the paraphrased version of the given math problem and nothing

# Run Inference of train data

In [79]:
output_pp = llm.generate(prompts_pp, sampling_params=sampling_params_pp)

Processed prompts:   0%|          | 0/10189 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/…

WARNING 04-25 17:56:54 [scheduler.py:1769] Sequence group 11822 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=151
WARNING 04-25 18:06:33 [scheduler.py:1769] Sequence group 13960 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=201
WARNING 04-25 18:17:59 [scheduler.py:1769] Sequence group 16506 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=251
WARNING 04-25 18:30:07 [schedul

# Inference on Test Data

In [80]:
prompts_pp_test = [prompt_template_pp.format(problem=q) for q in test["Question"]]
output_pp_test = llm.generate(prompts_pp_test, sampling_params=sampling_params_pp)

Processed prompts:   0%|          | 0/3044 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s…

WARNING 04-25 18:42:49 [scheduler.py:1769] Sequence group 21917 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=351


In [81]:
train["Question_pp"] = [o.outputs[0].text.strip() for o in output_pp]
test["Question_pp"] = [o.outputs[0].text.strip() for o in output_pp_test]

display(train.head())
display(test.head())

,Question,label,Question_pp
0,A solitaire game is played as follows. Six di...,3,A solitaire game uses six pairs of unique tile...
1,2. The school table tennis championship was he...,5,In a school table tennis championship using th...
2,"Given that $x, y,$ and $z$ are real numbers th...",0,"Three real numbers x, y, and z satisfy a serie..."
3,$25 \cdot 22$ Given three distinct points $P\l...,1,"Using three given points P, Q, and R, which ha..."
4,I am thinking of a five-digit number composed ...,5,Consider a five-digit number made up of even d...


,id,Question,Question_pp
0,0,b'Solve 0 = -i - 91*i - 1598*i - 64220 for i.\n',Find the value of 'i' that satisfies the equat...
1,1,Galperin G.A.\n\nA natural number $N$ is 999.....,We are looking for a number consisting of $k$ ...
2,2,Example 7 Calculate $\frac{1}{2 \sqrt{1}+\sqrt...,Compute the sum of the following series:\n\n1/...
3,3,"If $A$, $B$, and $C$ represent three distinct ...","Three digits, A, B, and C, each distinct and r..."
4,4,2. Calculate $1+12+123+1234+12345+123456+12345...,"Find the hundreds digit of the sum of 1, 12, 1..."


In [85]:
print("Actual:")
print(test["Question"].iloc[3])
print("\nParaphrased:")
print(test["Question_pp"].iloc[3])

Actual:
If $A$, $B$, and $C$ represent three distinct digits from 1 to 9 and they satisfy the following equations, what is the value of the sum $A+B+C$? (In the equation below, $AA$ represents a two-digit number both of whose digits are $A$.) $$A+B=C$$$$AA-B=2\times C$$$$C\times B=AA+A$$

Paraphrased:
Three digits, A, B, and C, each distinct and ranging from 1 to 9, adhere to the following relationships: A + B equals C, AA minus B equals twice C, and C times B equals AA plus A.  Determine the sum of A, B, and C.


# Save Paraphrased Data

In [84]:
train.to_csv("train_pp.csv", index=False)
test.to_csv("test_pp.csv", index=False)